In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import ensemble, preprocessing
from catboost import CatBoostClassifier, Pool


In [2]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [3]:
def train_test():
    
    train = pd.read_csv('./data/train.csv')
    test = pd.read_csv('./data/test.csv')
    print(train.shape), print(test.shape)
    
    ind_train = train['SK_ID_CURR']
    ind_test = test['SK_ID_CURR']
    
    new_df = train.append(test).reset_index()
    
    avg = [col for col in new_df.columns if 'AVG' in col]
    new_df['AVG_MEAN'] =  new_df[avg].sum(axis=1)
    new_df['AVG_SUM'] =  new_df[avg].sum(axis=1)
    new_df['AVG_STD'] =  new_df[avg].std(axis=1)
    new_df['AVG_MEDIAN'] =  new_df[avg].median(axis=1)
    
    mode = [col for col in new_df.columns if 'MODE' in col]
    new_df['MODE_MEAN'] =  new_df[mode].sum(axis=1)
    new_df['MODE_SUM'] =  new_df[mode].sum(axis=1)
    new_df['MODE_STD'] =  new_df[mode].std(axis=1)
    new_df['MODE_MEDIAN'] =  new_df[mode].median(axis=1)
    
    medi = [col for col in new_df.columns if 'MEDI' in col]
    new_df['MEDI_MEAN'] =  new_df[medi].sum(axis=1)
    new_df['MEDI_SUM'] =  new_df[medi].sum(axis=1)
    new_df['MEDI_STD'] =  new_df[medi].std(axis=1)
    new_df['MEDI_MEDIAN'] =  new_df[medi].median(axis=1)
    
    new_df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    new_df['AGE_CLIENT']                = new_df['DAYS_BIRTH']/-365
    new_df['EMPLOYED_YEAR']             = new_df['DAYS_EMPLOYED']/-365
    new_df['_REGISTRATION_YEAR']        = new_df['DAYS_REGISTRATION']/-365
    new_df['ID_PUBLISH_YEAR']           = new_df['DAYS_ID_PUBLISH'] / -365
    new_df['RATIO_CHILD_MEMBERS_FAM']   = new_df['CNT_CHILDREN'] /new_df['CNT_FAM_MEMBERS']
    new_df['RATIO_INCOME_MEMBERS_FAM']  = new_df['AMT_INCOME_TOTAL'] /new_df['CNT_FAM_MEMBERS']
    new_df['RATIO_INCOME_CREDIT']       = new_df['AMT_INCOME_TOTAL'] /new_df['AMT_CREDIT']
    new_df['RATIO_INCOME_ANNUITY']      = new_df['AMT_INCOME_TOTAL'] /new_df['AMT_ANNUITY']
    new_df['RATIO_PRICE_INCOME']        = new_df['AMT_GOODS_PRICE'] /new_df['AMT_INCOME_TOTAL']
    new_df['RATIO_PRICE_CREDIT']        = new_df['AMT_GOODS_PRICE'] /new_df['AMT_CREDIT']
    new_df['EXT_SCORE_SUM']             = new_df['EXT_SOURCE_1'] +new_df['EXT_SOURCE_2'] +new_df['EXT_SOURCE_3']
    new_df['EXT_SCORE_MEAN']            = (new_df['EXT_SOURCE_1'] +new_df['EXT_SOURCE_2'] +new_df['EXT_SOURCE_3'])/3
    new_df['OBS_90_CNT_SOCIAL_CIRCLE_SUM']  = new_df['OBS_30_CNT_SOCIAL_CIRCLE'] +new_df['OBS_60_CNT_SOCIAL_CIRCLE'] 
    new_df['OBS_90_CNT_SOCIAL_CIRCLE_MEAN']  = (new_df['OBS_30_CNT_SOCIAL_CIRCLE'] +new_df['OBS_60_CNT_SOCIAL_CIRCLE'])/2 
    new_df['DEF_90_CNT_SOCIAL_CIRCLE_MEAN']  = (new_df['DEF_60_CNT_SOCIAL_CIRCLE'] +new_df['DEF_30_CNT_SOCIAL_CIRCLE'])/2 
    new_df['DEF_90_CNT_SOCIAL_CIRCLE_SUM']  = new_df['DEF_60_CNT_SOCIAL_CIRCLE'] +new_df['DEF_30_CNT_SOCIAL_CIRCLE']    
    
    flag_doc_col = [col for col in new_df.columns if 'FLAG_DOCUMENT' in col]
    new_df['FLAG_DOC_MEAN'] =  train[flag_doc_col].mean(axis=1)
    new_df['FLAG_DOC_SUM'] =  train[flag_doc_col].sum(axis=1)
    
    new_df, col = one_hot_encoder(new_df)
    
    train = new_df.loc[new_df['SK_ID_CURR'].isin(ind_train)]
    test = new_df.loc[new_df['SK_ID_CURR'].isin(ind_test)]
    print(train.shape), print(test.shape)
    return train, test
    

In [4]:
train,test = train_test()

(307511, 122)
(48744, 121)


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(307511, 293)
(48744, 293)


In [5]:
def hist_mean_median(series):
    print('HISTOGRAM',series.hist())
    print('mean:',series.mean())
    print('median:',series.median())
    print('std:',series.std())
    print('min:',series.min())
    print('max:',series.max())
    print('sum:',series.sum())

In [6]:
def bureau_bb():
    bureau_balance = pd.read_csv('./data/bureau_balance.csv')
    bureau = pd.read_csv('./data/bureau.csv')
    print(bureau.shape)
    bureau_balance, cat_bb = one_hot_encoder(bureau_balance)
    bb_agg = {
        'MONTHS_BALANCE': ['median','min', 'max'],
             'STATUS_0':      ['sum','mean'],
             'STATUS_1':      ['sum','mean'],
             'STATUS_2':       ['sum','mean'],
             'STATUS_3':       ['sum','mean'],
             'STATUS_4':   ['sum','mean'],
             'STATUS_5':      ['sum','mean'],
             'STATUS_C':          ['sum','mean'],
             'STATUS_X':      ['sum','mean'],
             'STATUS_nan':      ['sum','mean']}
      
    bureau_balance_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(bb_agg)
    bureau_balance_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in  bureau_balance_agg.columns.tolist()])
    bureau_balance_agg = bureau_balance_agg.reset_index()
    bureau = bureau.merge(bureau_balance_agg, how='left',on='SK_ID_BUREAU').drop(['SK_ID_BUREAU'],axis=1)


    bureau, cat_b = one_hot_encoder(bureau)
    b_agg = {'DAYS_CREDIT':              ['median'],
             'CREDIT_DAY_OVERDUE':       ['median','min', 'max'],
             'DAYS_CREDIT_ENDDATE':      ['median'],
             'DAYS_ENDDATE_FACT':        ['median'],
             'DAYS_CREDIT_UPDATE':       ['median'],
             'AMT_CREDIT_MAX_OVERDUE':   ['min','max'],
             'CNT_CREDIT_PROLONG':       ['sum','mean','min','max'],
             'AMT_CREDIT_SUM':           ['min','mean','max'],
             'AMT_CREDIT_SUM_DEBT':      ['min','mean','max'],
             'AMT_CREDIT_SUM_LIMIT':     ['min','mean','max'],
             'AMT_CREDIT_SUM_OVERDUE':   ['min','mean','max'],
             'MONTHS_BALANCE_MEDIAN':    ['median'],
             'MONTHS_BALANCE_MIN':       ['min','median','max'],
             'MONTHS_BALANCE_MIN':       ['min','median','max'],
             'AMT_ANNUITY':              ['min','mean','max'],
            }
    cat_b_agg = {}
    
    for col in cat_b: 
        cat_b_agg[col] = ['mean'] 
        
    for col in cat_bb:
        cat_b_agg[col+'_SUM'] = ['mean'] 
        cat_b_agg[col+'_MEAN'] = ['mean'] 
        
         
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**b_agg,**cat_b_agg}) 
    bureau_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in  bureau_agg.columns.tolist()])
    bureau_agg = bureau_agg.reset_index()
    
    print(bureau_agg.shape)
    return bureau_agg

In [7]:
bureau = bureau_bb()

(1716428, 17)
(305811, 77)


In [8]:
train = pd.merge(train,bureau,how='left',on='SK_ID_CURR')
test = pd.merge(test,bureau,how='left',on='SK_ID_CURR')

In [9]:
def previous_application():
    previous_application = pd.read_csv('./data/previous_application.csv')
    print(previous_application.shape)
    previous_application , cat_pr = one_hot_encoder(previous_application)
    
    previous_application['RATE_DOWN_PAYMENT'][previous_application['RATE_DOWN_PAYMENT']<0] = np.nan
    previous_application['AMT_DOWN_PAYMENT'][previous_application['AMT_DOWN_PAYMENT']<0] = np.nan
    previous_application['DAYS_TERMINATION'][previous_application['DAYS_TERMINATION'] == 365243] = np.nan
    previous_application['DAYS_LAST_DUE'][previous_application['DAYS_LAST_DUE'] == 365243] = np.nan
    previous_application['DAYS_FIRST_DUE'][previous_application['DAYS_FIRST_DUE'] == 365243] = np.nan
    previous_application['DAYS_FIRST_DRAWING'][previous_application['DAYS_FIRST_DRAWING'] == 365243] = np.nan
    
    
    
    pa_agg ={'AMT_ANNUITY':              ['median','min','max'],
             'AMT_APPLICATION':          ['median','min','max'],
             'AMT_CREDIT':               ['median','min','max'],
             'AMT_DOWN_PAYMENT':         ['median','min','max'],
             'AMT_GOODS_PRICE':          ['median','min','max'],
             'HOUR_APPR_PROCESS_START':  ['mean','min','max'],
             'NFLAG_LAST_APPL_IN_DAY':   ['sum'],
             'RATE_DOWN_PAYMENT':        ['mean','min','max','sum'],
             'RATE_INTEREST_PRIMARY':    ['mean','min','max','sum'],
             'RATE_INTEREST_PRIVILEGED': ['mean','min','max','sum'],
             'DAYS_DECISION':            ['median','min','max'],
             'CNT_PAYMENT':              ['median','min','max'],
             'DAYS_FIRST_DRAWING':       ['median','min','max'],
             'DAYS_FIRST_DUE':           ['median','min','max'],
             'DAYS_LAST_DUE':            ['median','min','max'],
             'DAYS_TERMINATION':         ['median','min','max'],
             'NFLAG_INSURED_ON_APPROVAL':['sum']}
    
    cat_agg = {}
    for cat in cat_pr:
        cat_agg[cat] = ['mean']
    #previous_application_sk_id = previous_application[['SK_ID_CURR','SK_ID_PREV']]
    #previous_application_sk_id = previous_application_sk_id.gr
    previous_application_agg = previous_application.groupby('SK_ID_CURR').agg({**pa_agg, **cat_agg})
    previous_application_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in  previous_application_agg.columns.tolist()])
    previous_application_agg = previous_application_agg.reset_index()
    #previous_application_agg = previous_application_agg.merge(previous_application_sk_id,how='left',on='SK_ID_CURR')
    print(previous_application_agg.shape)
    #previous_application_agg = previous_application_agg.groupby(['SK_ID_CURR']).mean()
        
    return previous_application_agg

In [10]:
previous_application = previous_application()

(1670214, 37)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

(338857, 210)


In [11]:
train = pd.merge(train,previous_application,how='left',on='SK_ID_CURR')
test = pd.merge(test,previous_application,how='left',on='SK_ID_CURR')

In [12]:
def POS_CASH_balance():
    POS_CASH_balance = pd.read_csv('./data/POS_CASH_balance.csv')
    print(POS_CASH_balance.shape)
    POS_CASH_balance , cat_pc_b = one_hot_encoder(POS_CASH_balance)
    

    
    pc_b_agg = {'MONTHS_BALANCE':  ['median','min','max'],
        'CNT_INSTALMENT':  ['median','min','max'],
             'CNT_INSTALMENT_FUTURE':  ['median','min','max'],
             'SK_DPD':                 ['median','min','max'],
             'SK_DPD_DEF':             ['median','min','max'],
            }
    cat_agg = {}
    for cat in cat_pc_b:
        cat_agg[cat] = ['mean']
 
    
    POS_CASH_balance_agg = POS_CASH_balance.groupby(['SK_ID_CURR']).agg({**pc_b_agg, **cat_agg})
    POS_CASH_balance_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in  POS_CASH_balance_agg.columns.tolist()])
    POS_CASH_balance_agg = POS_CASH_balance_agg.reset_index()
    print(POS_CASH_balance_agg.shape)
    return POS_CASH_balance_agg

    

In [13]:
POS_CASH_balance = POS_CASH_balance()

(10001358, 8)
(337252, 26)


In [14]:
train = pd.merge(train,POS_CASH_balance,how='left',on='SK_ID_CURR')
test = pd.merge(test,POS_CASH_balance,how='left',on='SK_ID_CURR')

In [15]:
def credit_card_balance():
    credit_card_balance = pd.read_csv('./data/credit_card_balance.csv')
    credit_card_balance , cat_cc_b = one_hot_encoder(credit_card_balance)
    print(credit_card_balance.shape)
    credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'][credit_card_balance['AMT_DRAWINGS_ATM_CURRENT']<0] = np.nan
    credit_card_balance['AMT_DRAWINGS_CURRENT'][credit_card_balance['AMT_DRAWINGS_CURRENT']<0] = np.nan
    
    cc_agg =     { 'MONTHS_BALANCE':          ['median','min','max'],
                 'AMT_BALANCE':               ['median','min','max'],
                 'AMT_CREDIT_LIMIT_ACTUAL':   ['median'],
                 'AMT_DRAWINGS_ATM_CURRENT':  ['median'],
                 'AMT_DRAWINGS_CURRENT':      ['mean'],
                 'AMT_DRAWINGS_OTHER_CURRENT':['mean'],
                 'AMT_DRAWINGS_POS_CURRENT':  ['mean'],
                 'AMT_INST_MIN_REGULARITY':   ['mean'],
                 'AMT_PAYMENT_CURRENT':       ['median'],
                 'AMT_PAYMENT_TOTAL_CURRENT': ['mean'],
                 'AMT_RECEIVABLE_PRINCIPAL':  ['mean'],
                 'AMT_RECIVABLE':             ['mean'],
                 'AMT_TOTAL_RECEIVABLE':      ['mean'],
                 'CNT_DRAWINGS_ATM_CURRENT':  ['mean','min','max'],
                 'CNT_DRAWINGS_CURRENT':      ['mean','min','max'],
                 'CNT_DRAWINGS_OTHER_CURRENT':['mean','min','max'],
                 'CNT_DRAWINGS_POS_CURRENT':  ['mean','min','max'],
                 'CNT_INSTALMENT_MATURE_CUM': ['median','min','max'],
                 'SK_DPD':                    ['mean','min','max'],
                 'SK_DPD_DEF':                ['mean','min','max']}
    
    cat_agg = {}
    for cat in cat_cc_b:
            cat_agg[cat] = ['mean']
    
    credit_card_balance_agg = credit_card_balance.groupby('SK_ID_CURR').agg({**cc_agg, **cat_agg})
    credit_card_balance_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in  credit_card_balance_agg.columns.tolist()])
    credit_card_balance_agg = credit_card_balance_agg.reset_index()
    print(credit_card_balance_agg.shape)
    return credit_card_balance_agg
    

    
    

In [16]:
credit_card_balance = credit_card_balance()

(3840312, 30)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(103558, 47)


In [17]:
train = pd.merge(train,credit_card_balance,how='left',on='SK_ID_CURR')
test = pd.merge(test,credit_card_balance,how='left',on='SK_ID_CURR')

In [18]:
def installments_payments():
    installments_payments = pd.read_csv('./data/installments_payments.csv')
    print(installments_payments.shape)
    ip_agg =     {   'NUM_INSTALMENT_VERSION':['median','min','max'],
                 'NUM_INSTALMENT_NUMBER':     ['median','min','max'],
                 'DAYS_INSTALMENT':           ['min','max'],
                 'DAYS_ENTRY_PAYMENT':        ['min','max'],
                 'AMT_INSTALMENT':            ['median'],
                 'AMT_PAYMENT':               ['median'],}
    installments_payments_agg = installments_payments.groupby('SK_ID_CURR').agg(ip_agg)
    installments_payments_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in  installments_payments_agg.columns.tolist()])
    installments_payments_agg = installments_payments_agg.reset_index()
    print(installments_payments_agg.shape)
    return installments_payments_agg
    

In [19]:
installments_payments = installments_payments()

(13605401, 8)
(339587, 13)


In [20]:
train = pd.merge(train,installments_payments,how='left',on='SK_ID_CURR')
test = pd.merge(test,installments_payments,how='left',on='SK_ID_CURR')

In [21]:
labels = train['TARGET']
idx = test['SK_ID_CURR']
train = train.drop(['index', 'SK_ID_CURR', 'TARGET'],axis=1) 
test = test.drop(['index', 'SK_ID_CURR','TARGET'],axis=1) 

In [24]:
def nan(df):
    for column in df:
        if df[column].isnull().any():
               if df[column].dtypes == 'object':
                    values = df[column].value_counts().keys().tolist()
                    counts = df[column].value_counts().tolist()
                    if len(counts)!=0:
                        maxx = max(counts)
                        ind  = counts.index(maxx)
                        df[column] = df[column].fillna(values[ind])
               else:
                        df[column] = df[column].fillna(df[column].mean())
    return df
    del values, counts, maxx, ind, df



In [25]:
train = nan(train)
test = nan(test)

In [32]:
train

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,NUM_INSTALMENT_VERSION_MAX,NUM_INSTALMENT_NUMBER_MEDIAN,NUM_INSTALMENT_NUMBER_MIN,NUM_INSTALMENT_NUMBER_MAX,DAYS_INSTALMENT_MIN,DAYS_INSTALMENT_MAX,DAYS_ENTRY_PAYMENT_MIN,DAYS_ENTRY_PAYMENT_MAX,AMT_INSTALMENT_MEDIAN,AMT_PAYMENT_MEDIAN
0,24700.5,406597.5,351000.0,202500.000,0.000,0.000000,0.000000,0.000000,0.000000,1.000000,...,2.000000,10.000000,1.000000,19.000000,-565.00000,-25.00000,-587.000000,-49.000000,9251.775000,9251.775000
1,35698.5,1293502.5,1129500.0,270000.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,5.000000,1.000000,12.000000,-2310.00000,-536.00000,-2324.000000,-544.000000,64275.615000,64275.615000
2,6750.0,135000.0,135000.0,67500.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,2.000000,1.000000,3.000000,-784.00000,-724.00000,-795.000000,-727.000000,5357.250000,5357.250000
3,29686.5,312682.5,297000.0,135000.000,0.007,0.006402,0.267395,0.265474,0.034362,1.899974,...,2.000000,4.000000,1.000000,10.000000,-545.00000,-11.00000,-575.000000,-12.000000,29027.520000,29027.520000
4,21865.5,513000.0,513000.0,121500.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,7.000000,1.000000,17.000000,-2326.00000,-14.00000,-2318.000000,-14.000000,16037.640000,16037.640000
5,27517.5,490495.5,454500.0,99000.000,0.000,0.000000,0.000000,1.000000,0.000000,1.000000,...,2.000000,5.000000,1.000000,10.000000,-2491.00000,-69.00000,-2519.000000,-82.000000,17876.115000,17876.115000
6,41301.0,1560726.0,1395000.0,171000.000,0.000,0.000000,1.000000,1.000000,0.000000,2.000000,...,1.000000,4.000000,1.000000,12.000000,-2908.00000,-43.00000,-2914.000000,-58.000000,8996.760000,8996.760000
7,42075.0,1530000.0,1530000.0,360000.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,5.500000,1.000000,10.000000,-1039.00000,-769.00000,-1069.000000,-774.000000,27463.410000,27463.410000
8,33826.5,1019610.0,913500.0,112500.000,0.000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,23.500000,1.000000,76.000000,-2477.00000,-37.00000,-2477.000000,-37.000000,9000.000000,9000.000000
9,20250.0,405000.0,405000.0,135000.000,0.007,0.006402,0.267395,0.265474,0.034362,1.899974,...,4.000000,7.500000,1.000000,23.000000,-1641.00000,-142.00000,-1645.000000,-151.000000,11057.535000,11057.535000


In [26]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
# Some useful parameters which will come in handy later on
SEED = 4 # for reproducibility
NFOLDS = 5# set folds for out-of-fold prediction
kf = KFold(n_splits= NFOLDS)
i=0
for train_index, test_index in kf.split(train):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
#for i, (train_index, test_index) in enumerate(kf):
    #print(i,train_index, test_index)

TRAIN: 246008 TEST: 61503
TRAIN: 246009 TEST: 61502
TRAIN: 246009 TEST: 61502
TRAIN: 246009 TEST: 61502
TRAIN: 246009 TEST: 61502


In [29]:
ntrain=train.shape[0]
ntest=test.shape[0]
print(ntrain, ntest)

307511 48744


In [35]:
pp = {'verbosity': 1,
 'reg_lambda': 0.5780000000000001,
 'reg_alpha': 1.09,
 'objective': 'binary',
 'num_leaves': 491,
 'n_estimators': 4000,
 'min_data_in_leaf': 138,
 'min_child_weight': 0.019869999999999964,
 'metric': 'auc',
 'max_depth': 15,
 'learning_rate': 0.006,
 'feature_fraction': 0.17100000000000007,
 'boosting_type': 'gbdt',
 'bagging_seed': 1,
 'bagging_fraction': 0.5350000000000001
}

In [36]:
pred_train = np.zeros((ntrain,) )
score = 0
from sklearn import metrics
import lightgbm
i = 1
for train_index, test_index in kf.split(train):
    train_data = lightgbm.Dataset(train.iloc[train_index], label=labels.iloc[train_index])
    val_train_data = lightgbm.Dataset(train.iloc[test_index], label=labels.iloc[test_index])
    test_data = lightgbm.Dataset(test)
    
    
    
    model = lightgbm.train(pp,
                           train_data,
                           valid_sets=val_train_data,
                           num_boost_round=4000,
                           early_stopping_rounds=100)
    
    model.save_model('model'+str(i)+'.txt')
    pr=model.predict(train.iloc[test_index])
    pred_train[test_index]=pr
    err=metrics.roc_auc_score(labels.iloc[test_index], pr)
    score += err*(1/NFOLDS) 
    print("Fold", i, "Точность работы на тестовых данных:",  err)
    i+=1

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.633463
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.671338
[3]	valid_0's auc: 0.690417
[4]	valid_0's auc: 0.696615
[5]	valid_0's auc: 0.716065
[6]	valid_0's auc: 0.718305
[7]	valid_0's auc: 0.719985
[8]	valid_0's auc: 0.721818
[9]	valid_0's auc: 0.723315
[10]	valid_0's auc: 0.729092
[11]	valid_0's auc: 0.733966
[12]	valid_0's auc: 0.735721
[13]	valid_0's auc: 0.735632
[14]	valid_0's auc: 0.739455
[15]	valid_0's auc: 0.742438
[16]	valid_0's auc: 0.74683
[17]	valid_0's auc: 0.747675
[18]	valid_0's auc: 0.747024
[19]	valid_0's auc: 0.747801
[20]	valid_0's auc: 0.749382
[21]	valid_0's auc: 0.748884
[22]	valid_0's auc: 0.748755
[23]	valid_0's auc: 0.747991
[24]	valid_0's auc: 0.749152
[25]	valid_0's auc: 0.749567
[26]	valid_0's auc: 0.749746
[27]	valid_0's auc: 0.749649
[28]	valid_0's auc: 0.750252
[29]	valid_0's auc: 0.751653
[30]	valid_0's auc: 0.75177
[31]	valid_0's auc: 0.751666
[32]	valid_0's auc: 0.751109
[33]	valid_0's auc: 

[277]	valid_0's auc: 0.770416
[278]	valid_0's auc: 0.770464
[279]	valid_0's auc: 0.770436
[280]	valid_0's auc: 0.770526
[281]	valid_0's auc: 0.770569
[282]	valid_0's auc: 0.770664
[283]	valid_0's auc: 0.770717
[284]	valid_0's auc: 0.770818
[285]	valid_0's auc: 0.77082
[286]	valid_0's auc: 0.770823
[287]	valid_0's auc: 0.770855
[288]	valid_0's auc: 0.770865
[289]	valid_0's auc: 0.770931
[290]	valid_0's auc: 0.770884
[291]	valid_0's auc: 0.770956
[292]	valid_0's auc: 0.770956
[293]	valid_0's auc: 0.770965
[294]	valid_0's auc: 0.770953
[295]	valid_0's auc: 0.770955
[296]	valid_0's auc: 0.770968
[297]	valid_0's auc: 0.770978
[298]	valid_0's auc: 0.771019
[299]	valid_0's auc: 0.771044
[300]	valid_0's auc: 0.77106
[301]	valid_0's auc: 0.771184
[302]	valid_0's auc: 0.771269
[303]	valid_0's auc: 0.771358
[304]	valid_0's auc: 0.771329
[305]	valid_0's auc: 0.771358
[306]	valid_0's auc: 0.77138
[307]	valid_0's auc: 0.771459
[308]	valid_0's auc: 0.77148
[309]	valid_0's auc: 0.771498
[310]	valid_0'

[552]	valid_0's auc: 0.77768
[553]	valid_0's auc: 0.777719
[554]	valid_0's auc: 0.777745
[555]	valid_0's auc: 0.777763
[556]	valid_0's auc: 0.777797
[557]	valid_0's auc: 0.777797
[558]	valid_0's auc: 0.777866
[559]	valid_0's auc: 0.777865
[560]	valid_0's auc: 0.777873
[561]	valid_0's auc: 0.777897
[562]	valid_0's auc: 0.777896
[563]	valid_0's auc: 0.777884
[564]	valid_0's auc: 0.777908
[565]	valid_0's auc: 0.77792
[566]	valid_0's auc: 0.777916
[567]	valid_0's auc: 0.777932
[568]	valid_0's auc: 0.777947
[569]	valid_0's auc: 0.777961
[570]	valid_0's auc: 0.77797
[571]	valid_0's auc: 0.777971
[572]	valid_0's auc: 0.77801
[573]	valid_0's auc: 0.778031
[574]	valid_0's auc: 0.778032
[575]	valid_0's auc: 0.778054
[576]	valid_0's auc: 0.778082
[577]	valid_0's auc: 0.778085
[578]	valid_0's auc: 0.778083
[579]	valid_0's auc: 0.778083
[580]	valid_0's auc: 0.778089
[581]	valid_0's auc: 0.77809
[582]	valid_0's auc: 0.778098
[583]	valid_0's auc: 0.778087
[584]	valid_0's auc: 0.778101
[585]	valid_0's

[827]	valid_0's auc: 0.781287
[828]	valid_0's auc: 0.781294
[829]	valid_0's auc: 0.781294
[830]	valid_0's auc: 0.781303
[831]	valid_0's auc: 0.7813
[832]	valid_0's auc: 0.781307
[833]	valid_0's auc: 0.781308
[834]	valid_0's auc: 0.781324
[835]	valid_0's auc: 0.781337
[836]	valid_0's auc: 0.781358
[837]	valid_0's auc: 0.781359
[838]	valid_0's auc: 0.781373
[839]	valid_0's auc: 0.781379
[840]	valid_0's auc: 0.781388
[841]	valid_0's auc: 0.781391
[842]	valid_0's auc: 0.781387
[843]	valid_0's auc: 0.781381
[844]	valid_0's auc: 0.781407
[845]	valid_0's auc: 0.781423
[846]	valid_0's auc: 0.781421
[847]	valid_0's auc: 0.781431
[848]	valid_0's auc: 0.78145
[849]	valid_0's auc: 0.781456
[850]	valid_0's auc: 0.781459
[851]	valid_0's auc: 0.781474
[852]	valid_0's auc: 0.781493
[853]	valid_0's auc: 0.781504
[854]	valid_0's auc: 0.781514
[855]	valid_0's auc: 0.781522
[856]	valid_0's auc: 0.781529
[857]	valid_0's auc: 0.781535
[858]	valid_0's auc: 0.781543
[859]	valid_0's auc: 0.78154
[860]	valid_0'

...

[1364]	valid_0's auc: 0.785596
[1365]	valid_0's auc: 0.78559
[1366]	valid_0's auc: 0.785594
[1367]	valid_0's auc: 0.785607
[1368]	valid_0's auc: 0.785603
[1369]	valid_0's auc: 0.785615
[1370]	valid_0's auc: 0.785618
[1371]	valid_0's auc: 0.785625
[1372]	valid_0's auc: 0.785628
[1373]	valid_0's auc: 0.785623
[1374]	valid_0's auc: 0.785623
[1375]	valid_0's auc: 0.785612
[1376]	valid_0's auc: 0.785605
[1377]	valid_0's auc: 0.785596
[1378]	valid_0's auc: 0.785603
[1379]	valid_0's auc: 0.785598
[1380]	valid_0's auc: 0.7856
[1381]	valid_0's auc: 0.785597
[1382]	valid_0's auc: 0.785609
[1383]	valid_0's auc: 0.785608
[1384]	valid_0's auc: 0.785628
[1385]	valid_0's auc: 0.785626
[1386]	valid_0's auc: 0.785626
[1387]	valid_0's auc: 0.785628
[1388]	valid_0's auc: 0.785635
[1389]	valid_0's auc: 0.785633
[1390]	valid_0's auc: 0.785633
[1391]	valid_0's auc: 0.785634
[1392]	valid_0's auc: 0.785636
[1393]	valid_0's auc: 0.785643
[1394]	valid_0's auc: 0.785638
[1395]	valid_0's auc: 0.785648
[1396]	vali

[1630]	valid_0's auc: 0.786366
[1631]	valid_0's auc: 0.786373
[1632]	valid_0's auc: 0.786372
[1633]	valid_0's auc: 0.786378
[1634]	valid_0's auc: 0.786379
[1635]	valid_0's auc: 0.786391
[1636]	valid_0's auc: 0.786392
[1637]	valid_0's auc: 0.786396
[1638]	valid_0's auc: 0.786388
[1639]	valid_0's auc: 0.786389
[1640]	valid_0's auc: 0.78641
[1641]	valid_0's auc: 0.786424
[1642]	valid_0's auc: 0.786427
[1643]	valid_0's auc: 0.786423
[1644]	valid_0's auc: 0.786437
[1645]	valid_0's auc: 0.786435
[1646]	valid_0's auc: 0.786436
[1647]	valid_0's auc: 0.786442
[1648]	valid_0's auc: 0.786438
[1649]	valid_0's auc: 0.786437
[1650]	valid_0's auc: 0.786448
[1651]	valid_0's auc: 0.786454
[1652]	valid_0's auc: 0.786458
[1653]	valid_0's auc: 0.78646
[1654]	valid_0's auc: 0.786462
[1655]	valid_0's auc: 0.78646
[1656]	valid_0's auc: 0.786464
[1657]	valid_0's auc: 0.78646
[1658]	valid_0's auc: 0.786457
[1659]	valid_0's auc: 0.786464
[1660]	valid_0's auc: 0.786464
[1661]	valid_0's auc: 0.786467
[1662]	valid

[1896]	valid_0's auc: 0.786757
[1897]	valid_0's auc: 0.786753
[1898]	valid_0's auc: 0.786766
[1899]	valid_0's auc: 0.786776
[1900]	valid_0's auc: 0.786781
[1901]	valid_0's auc: 0.786786
[1902]	valid_0's auc: 0.786787
[1903]	valid_0's auc: 0.786779
[1904]	valid_0's auc: 0.786793
[1905]	valid_0's auc: 0.786793
[1906]	valid_0's auc: 0.78679
[1907]	valid_0's auc: 0.786791
[1908]	valid_0's auc: 0.786791
[1909]	valid_0's auc: 0.786791
[1910]	valid_0's auc: 0.786794
[1911]	valid_0's auc: 0.786798
[1912]	valid_0's auc: 0.786784
[1913]	valid_0's auc: 0.786788
[1914]	valid_0's auc: 0.786792
[1915]	valid_0's auc: 0.786791
[1916]	valid_0's auc: 0.786793
[1917]	valid_0's auc: 0.786794
[1918]	valid_0's auc: 0.786792
[1919]	valid_0's auc: 0.786789
[1920]	valid_0's auc: 0.786794
[1921]	valid_0's auc: 0.786795
[1922]	valid_0's auc: 0.786801
[1923]	valid_0's auc: 0.7868
[1924]	valid_0's auc: 0.786802
[1925]	valid_0's auc: 0.786801
[1926]	valid_0's auc: 0.786805
[1927]	valid_0's auc: 0.786802
[1928]	vali

[2162]	valid_0's auc: 0.786928
[2163]	valid_0's auc: 0.786923
[2164]	valid_0's auc: 0.786934
[2165]	valid_0's auc: 0.786937
[2166]	valid_0's auc: 0.786934
[2167]	valid_0's auc: 0.786933
[2168]	valid_0's auc: 0.786933
[2169]	valid_0's auc: 0.786932
[2170]	valid_0's auc: 0.786939
[2171]	valid_0's auc: 0.786943
[2172]	valid_0's auc: 0.786936
[2173]	valid_0's auc: 0.786943
[2174]	valid_0's auc: 0.786932
[2175]	valid_0's auc: 0.786926
[2176]	valid_0's auc: 0.786926
[2177]	valid_0's auc: 0.786918
[2178]	valid_0's auc: 0.786919
[2179]	valid_0's auc: 0.786926
[2180]	valid_0's auc: 0.786923
[2181]	valid_0's auc: 0.786927
[2182]	valid_0's auc: 0.786935
[2183]	valid_0's auc: 0.786933
[2184]	valid_0's auc: 0.786938
[2185]	valid_0's auc: 0.786939
[2186]	valid_0's auc: 0.786937
[2187]	valid_0's auc: 0.78694
[2188]	valid_0's auc: 0.786943
[2189]	valid_0's auc: 0.786944
[2190]	valid_0's auc: 0.786949
[2191]	valid_0's auc: 0.786951
[2192]	valid_0's auc: 0.786954
[2193]	valid_0's auc: 0.786958
[2194]	va

[2427]	valid_0's auc: 0.78701
[2428]	valid_0's auc: 0.787006
[2429]	valid_0's auc: 0.787002
[2430]	valid_0's auc: 0.787014
[2431]	valid_0's auc: 0.787008
[2432]	valid_0's auc: 0.787011
[2433]	valid_0's auc: 0.787001
[2434]	valid_0's auc: 0.787001
[2435]	valid_0's auc: 0.787006
[2436]	valid_0's auc: 0.787008
[2437]	valid_0's auc: 0.787008
[2438]	valid_0's auc: 0.787003
[2439]	valid_0's auc: 0.787007
[2440]	valid_0's auc: 0.787001
[2441]	valid_0's auc: 0.787002
[2442]	valid_0's auc: 0.787005
[2443]	valid_0's auc: 0.787003
[2444]	valid_0's auc: 0.787002
[2445]	valid_0's auc: 0.787003
[2446]	valid_0's auc: 0.786999
[2447]	valid_0's auc: 0.787
[2448]	valid_0's auc: 0.787001
[2449]	valid_0's auc: 0.786992
[2450]	valid_0's auc: 0.786994
[2451]	valid_0's auc: 0.786993
Early stopping, best iteration is:
[2351]	valid_0's auc: 0.787062
Fold 5 Точность работы на тестовых данных: 0.7870624089097014


In [37]:
score

0.7856196800679234

In [39]:
wr=1./NFOLDS
predTest = np.zeros((len(test)))
for i in np.arange(NFOLDS):
    i+=1
    print('Loading model to predict... model'+str(i)+'.txt')
    # load model to predict
    bst = lightgbm.Booster(model_file='model'+str(i)+'.txt')
    predTr=bst.predict(test,num_iteration=bst.best_iteration)
    predTest=predTest+predTr*wr
    

Loading model to predict... model1.txt
Loading model to predict... model2.txt
Loading model to predict... model3.txt
Loading model to predict... model4.txt
Loading model to predict... model5.txt


In [41]:
preds = pd.DataFrame({"SK_ID_CURR": idx, "TARGET": predTest})
preds.to_csv('submission.csv', index=False)